In [72]:
#import panda library to read csv file
import pandas as pd

#training data file
df = pd.read_csv("Datasets/train.csv")

#testing data which we have to predict
df1 = pd.read_csv("Datasets/test.csv")

In [73]:
#structure of our data 
#these columns are feature vector
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [74]:
#training data
#only include those feature vector or columns by which our machine learning model predict
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis = 'columns',inplace = True)
df.head()


,Survived,Pclass,Sex,Age,Fare
0,0,3,male,22.0,7.2500
1,1,1,female,38.0,71.2833
2,1,3,female,26.0,7.9250
3,1,1,female,35.0,53.1000
4,0,3,male,35.0,8.0500


In [75]:
#testing data
#only include those feature vector or columns by which our machine learning model predict
df1.drop(['Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis = 'columns',inplace = True)
df1.head()

,PassengerId,Pclass,Sex,Age,Fare
0,892,3,male,34.5,7.8292
1,893,3,female,47.0,7.0000
2,894,2,male,62.0,9.6875
3,895,3,male,27.0,8.6625
4,896,3,female,22.0,12.2875


In [76]:
#seprating survived column
#store it in target vector
target = df.Survived
inputs = df.drop('Survived',axis = 'columns')

In [77]:
#OneHotEncoding on traing data
#for machine learning model we have to train over binary data instead of character
#converting gender type in form of 0 or 1
dummies = pd.get_dummies(inputs.Sex)
dummies.head()

,female,male
0,0,1
1,1,0
2,1,0
3,1,0
4,0,1


In [78]:
#OneHotEncoding on testing data
dummies1 = pd.get_dummies(df1.Sex)
dummies1.head()

,female,male
0,0,1
1,1,0
2,0,1
3,0,1
4,1,0


In [79]:
#concating our updated gender table which we get from onehotencoding
inputs = pd.concat([inputs,dummies],axis = 'columns')
inputs.drop('Sex',axis = 'columns',inplace = True)
inputs.head()

,Pclass,Age,Fare,female,male
0,3,22.0,7.2500,0,1
1,1,38.0,71.2833,1,0
2,3,26.0,7.9250,1,0
3,1,35.0,53.1000,1,0
4,3,35.0,8.0500,0,1


In [80]:
#store passenger id from testing csv file into pid vector and remove from df1
pid = df1.PassengerId
df1 = pd.concat([df1,dummies1],axis='columns')
df1.drop('Sex',axis='columns',inplace=True)
df1.drop('PassengerId',axis='columns',inplace=True)
df1.head()

,Pclass,Age,Fare,female,male
0,3,34.5,7.8292,0,1
1,3,47.0,7.0000,1,0
2,2,62.0,9.6875,0,1
3,3,27.0,8.6625,0,1
4,3,22.0,12.2875,1,0


In [81]:
#check which columns have NaN value
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [82]:
inputs.Age[:10]

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

In [83]:
#to remove NaN value from traing set we fill all those with mean of Age column
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.Age[:10]

0    22.000000
1    38.000000
2    26.000000
3    35.000000
4    35.000000
5    29.699118
6    54.000000
7     2.000000
8    27.000000
9    14.000000
Name: Age, dtype: float64

In [84]:
#to remove NaN value from testing set we fill all those with mean of Age column
df1.Age = df1.Age.fillna(df1.Age.mean())
df1.Fare = df1.Age.fillna(df1.Fare.mean())
df1.columns[df1.isna().any()]

Index([], dtype='object')

In [85]:
# storing values of feature vector and its already predicted value wheteher survive or not into X-train and y_train
X_train,y_train = inputs,target

In [86]:
#importing machine learning model Gaussian Naive Bayes classifier
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [87]:
#trainig of our model on train dataset
model.fit(X_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [88]:
#predicting value whether person survive or not and store it into survive vector
survive = model.predict(df1)

In [89]:
#store predicted value on test dataset into prediction.csv file
dict = {'PassengerId':pid,'Survived':survive}
submit = pd.DataFrame(dict)
submit.to_csv('prediction.csv',index = False)